In [7]:
import sys
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import json

In [9]:
load_dotenv()
load_dotenv('.env')

True

In [11]:

sports_data_api_key = os.getenv('SPORTS_DATA_API_KEY')
url = "https://api-nba-v1.p.rapidapi.com/players"
api_key_str = '?key='+sports_data_api_key
querystring = {"team":"9","season":"2023"}

headers = {
	"X-RapidAPI-Key": sports_data_api_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
response_nuggets = response.json()
print(json.dumps(response_nuggets, indent=4))

{
    "get": "players/",
    "parameters": {
        "team": "9",
        "season": "2023"
    },
    "errors": [],
    "results": 22,
    "response": [
        {
            "id": 383,
            "firstname": "Jamal",
            "lastname": "Murray",
            "birth": {
                "date": "1997-02-23",
                "country": "Canada"
            },
            "nba": {
                "start": 2016,
                "pro": 5
            },
            "height": {
                "feets": "6",
                "inches": "3",
                "meters": "1.9"
            },
            "weight": {
                "pounds": "215",
                "kilograms": "97.5"
            },
            "college": "Kentucky",
            "affiliation": "Kentucky/Canada",
            "leagues": {
                "standard": {
                    "jersey": 27,
                    "active": true,
                    "pos": "G"
                }
            }
        },
        {
            

In [13]:
nuggets_df = pd.json_normalize(response_nuggets['response']).reset_index(drop=True)
nuggets_df

,id,firstname,lastname,college,affiliation,birth.date,birth.country,nba.start,nba.pro,height.feets,...,leagues.standard.pos,leagues.vegas.jersey,leagues.vegas.active,leagues.vegas.pos,leagues.utah.jersey,leagues.utah.active,leagues.utah.pos,leagues.sacramento.jersey,leagues.sacramento.active,leagues.sacramento.pos
0,383,Jamal,Murray,Kentucky,Kentucky/Canada,1997-02-23,Canada,2016,5,6,...,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1055,Amida,Brimah,Connecticut,Connecticut/USA,1994-02-11,USA,2020,1,6,...,C,37.0,True,C,37.0,True,C,NaN,NaN,NaN
2,1014,Michael,Porter Jr.,Missouri,Missouri/USA,1998-06-29,USA,2019,2,6,...,F,1.0,True,F,NaN,NaN,NaN,NaN,NaN,NaN
3,2627,Zeke,Nnaji,Arizona,Arizona/USA,2001-01-09,USA,2020,1,6,...,F-C,22.0,True,F,NaN,NaN,NaN,NaN,NaN,NaN
4,3091,Braxton,Key,Virginia,Virginia/USA,1997-02-14,USA,2021,0,6,...,F,8.0,True,F,NaN,NaN,NaN,NaN,NaN,NaN
5,2814,Jay,Huff,Virginia,Virginia/USA,1997-08-25,USA,2021,0,None,...,F,30.0,True,C,NaN,NaN,NaN,30.0,True,C
6,2840,Jamorko,Pickett,Georgetown,Georgetown/USA,1997-12-24,USA,2021,0,6,...,F,45.0,True,F,NaN,NaN,NaN,NaN,NaN,NaN
7,3420,Christian,Braun,None,None,None,None,0,0,None,...,G,NaN,True,F,NaN,NaN,NaN,NaN,NaN,NaN
8,3498,Peyton,Watson,None,None,None,None,0,0,None,...,G,8.0,True,F,NaN,NaN,NaN,NaN,NaN,NaN
9,3441,Collin,Gillespie,None,None,None,None,0,0,None,...,G,21.0,True,G,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
nuggets_df_csv = nuggets_df.to_csv('Resources/Denver Nuggets.csv', index=False)

In [17]:
import time

data_frames = []

def get_player_stats(player_id):
    time.sleep(30)
    return pd.json_normalize(requests.get(url+"/statistics", headers=headers, params={"id":player_id, "season":"2023"}).json()['response'])

for id in nuggets_df['id']:
    df = get_player_stats(id)
    data_frames.append(df)

combined_df = pd.concat(data_frames, ignore_index=True)
combined_df

,points,pos,min,fgm,fga,fgp,ftm,fta,ftp,tpm,...,comment,player.id,player.firstname,player.lastname,team.id,team.name,team.nickname,team.code,team.logo,game.id
0,12,PG,18,5,11,45.5,1,1,100.0,1,...,None,383,Jamal,Murray,9,Denver Nuggets,Nuggets,DEN,https://upload.wikimedia.org/wikipedia/fr/thum...,12496
1,6,PG,18,3,6,50.0,0,0,0,0,...,None,383,Jamal,Murray,9,Denver Nuggets,Nuggets,DEN,https://upload.wikimedia.org/wikipedia/fr/thum...,12505
2,21,PG,22,6,11,54.5,8,10,80.0,1,...,None,383,Jamal,Murray,9,Denver Nuggets,Nuggets,DEN,https://upload.wikimedia.org/wikipedia/fr/thum...,12517
3,8,PG,29,3,11,27.3,1,2,50.0,1,...,None,383,Jamal,Murray,9,Denver Nuggets,Nuggets,DEN,https://upload.wikimedia.org/wikipedia/fr/thum...,12539
4,21,PG,34,8,13,61.5,2,2,100.0,3,...,None,383,Jamal,Murray,9,Denver Nuggets,Nuggets,DEN,https://upload.wikimedia.org/wikipedia/fr/thum...,12549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,25,C,41,12,21,57.1,0,0,0,1,...,None,279,Nikola,Jokic,9,Denver Nuggets,Nuggets,DEN,https://upload.wikimedia.org/wikipedia/fr/thum...,13964
1090,32,C,41,11,25,44.0,8,8,100.0,2,...,None,279,Nikola,Jokic,9,Denver Nuggets,Nuggets,DEN,https://upload.wikimedia.org/wikipedia/fr/thum...,13996
1091,16,C,39,5,13,38.5,6,6,100.0,0,...,None,279,Nikola,Jokic,9,Denver Nuggets,Nuggets,DEN,https://upload.wikimedia.org/wikipedia/fr/thum...,13997
1092,24,C,38,10,18,55.6,3,4,75.0,1,...,None,279,Nikola,Jokic,9,Denver Nuggets,Nuggets,DEN,https://upload.wikimedia.org/wikipedia/fr/thum...,13998


In [ ]:
nuggets_stats = combined_df.to_csv('Resources/Stats Denver Nuggets.csv', index=False)